In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model(r"artifacts/training/model.h5")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    repo_owner: str
    repo_name: str
    params_image_size: list
    params_batch_size: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri=self.config.evaluation.mlflow_uri,
            repo_owner=self.config.evaluation.repo_owner,
            repo_name=self.config.evaluation.repo_name,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub
from cnnClassifier import logger

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        logger.info(f"mlflow registry destination: {tracking_url_type_store} {self.config.mlflow_uri}")
        
        if tracking_url_type_store != "file":
            logger.info("wake up dagshub")
            dagshub.init(repo_owner=self.config.repo_owner, repo_name=self.config.repo_name, mlflow=True)
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model-v3")
            else:
                mlflow.keras.log_model(self.model, "model")

In [11]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-01-24 16:49:50,553: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-24 16:49:50,578: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-24 16:49:50,581: INFO: common: created directory at: artifacts]


Found 102 images belonging to 2 classes.
7/7 [==============================] - 31s 4s/step - loss: 20.8800 - accuracy: 0.5686
[2025-01-24 16:50:22,141: INFO: common: json file saved at: scores.json]
[2025-01-24 16:50:22,147: INFO: 435129321: mlflow registry destination: https https://dagshub.com/comboonsong/end-to-end-mlops-tutorial.mlflow]
[2025-01-24 16:50:22,151: INFO: 435129321: wake up dagshub]
[2025-01-24 16:50:22,196: INFO: tokens: Removed expired tokens from the token cache]


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\PiyabutInbunsong\miniconda3\envs\mlops-tutorial\lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=922c0f23-579c-432f-9d9c-d0b1f8662e88&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=61f4aeed1625e71f02bd537be4bf06c3064c19341780ae152d9c159b0cd1c328


[2025-01-24 16:50:29,609: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


...

[2025-01-24 16:50:36,987: INFO: helpers: Repository comboonsong/end-to-end-mlops-tutorial initialized!]


2025/01/24 16:50:40 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-01-24 16:50:46,626: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\PIYABU~1\AppData\Local\Temp\tmpuiy4eumq\model\data\model\assets
[2025-01-24 16:50:49,394: INFO: builder_impl: Assets written to: C:\Users\PIYABU~1\AppData\Local\Temp\tmpuiy4eumq\model\data\model\assets]


2025/01/24 16:51:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model-v2'.
2025/01/24 16:52:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model-v2, version 1
Created version '1' of model 'VGG16Model-v2'.


🏃 View run defiant-shad-448 at: https://dagshub.com/comboonsong/end-to-end-mlops-tutorial.mlflow/#/experiments/0/runs/50c1ac56aa58452d9506be3aa13ab8ec
🧪 View experiment at: https://dagshub.com/comboonsong/end-to-end-mlops-tutorial.mlflow/#/experiments/0
